Secure ETL & Data Engineering to Build a controlled data ingestion and preprocessing pipeline.  
  
Data-Simulated datasets  
sensor_logs.csv  
mission_profile.json  
maintenance_records.csv  
  
✔ Ingest multi-source data  
✔ Merge using Equipment ID & Timestamp  
✔ Handle missing sensor data  
Feature engineering:  
rolling averages  
risk indicators  
categorical encoding  

In [ ]:
%load_ext autoreload
%autoreload 2
# Do not run this cell unless running the code on a local notebook. This line is for auto-reloading the code in the notebook whenever you make changes to the code files. 
# If you're running this code in a Jupyter notebook/ Google Colab, you can uncomment the line to enable auto-reloading of your code files.

In [ ]:
%pip install pandas numpy scikit-learn matplotlib sqlite3 #ipykernel # Uncomment this line if running in a Jupyter notebook locally
# This line ensures that the necessary libraries are installed. If you're running this code in a Jupyter notebook/ Google Colab, you can uncomment the line to install the libraries directly from the notebook.

In [ ]:
import json
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# -----------------------
# 1) Load data
# -----------------------
sensor = pd.read_csv("./dataset/sensor_logs.csv")
sensor["timestamp"] = pd.to_datetime(sensor["timestamp"], errors="coerce")
sensor = sensor.dropna(subset=["timestamp"]).sort_values(["asset_id", "timestamp"])

mission = json.load(open("./dataset/mission_profile.json", "r", encoding="utf-8"))
mission = pd.json_normalize(mission if isinstance(mission, list) else [mission])

maint = pd.read_csv("./dataset/maintenance_records.csv")
maint["event_time"] = pd.to_datetime(maint["event_time"], errors="coerce")
maint = maint.dropna(subset=["event_time"]).sort_values(["asset_id", "event_time"])



In [ ]:
# -----------------------
# 2) Feature engineering
#    a) rolling averages (per asset)
# -----------------------
# Pick numeric sensor columns automatically (excluding id + time)
num_sensor_cols = [
    c for c in sensor.columns
    if c not in ["asset_id", "timestamp"] and pd.api.types.is_numeric_dtype(sensor[c])
]

# Rolling mean over last 5 rows (simple + minimal)
for c in num_sensor_cols:
    sensor[f"{c}_rollmean_5"] = (
        sensor.groupby("asset_id")[c]
        .rolling(window=5, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )



In [ ]:
# -----------------------
# 2b) risk indicators
# -----------------------
# Simple thresholds (change to match your columns)
THRESHOLDS = {"temperature": 85, "vibration": 4.5}

risk_flag_cols = []
for col, thr in THRESHOLDS.items():
    if col in sensor.columns:
        flag = f"risk_{col}_high"
        sensor[flag] = (sensor[col] > thr).astype(int)
        risk_flag_cols.append(flag)

sensor["risk_score"] = sensor[risk_flag_cols].sum(axis=1) if risk_flag_cols else 0



In [ ]:
# -----------------------
# 3) Join datasets
#    mission: plain left join
#    maintenance: "last maintenance event before this sensor timestamp"
# -----------------------
df = sensor.merge(mission, on="asset_id", how="left")

# merge_asof needs sorted frames
df = df.sort_values(["asset_id", "timestamp"])
maint = maint.sort_values(["asset_id", "event_time"])

# asof join per asset (minimal & correct)
out = []
for asset_id, g in df.groupby("asset_id", sort=False):
    m = maint[maint["asset_id"] == asset_id]
    if m.empty:
        out.append(g)
        continue
    out.append(
        pd.merge_asof(
            g.sort_values("timestamp"),
            m.sort_values("event_time"),
            left_on="timestamp",
            right_on="event_time",
            direction="backward",
        )
    )

df = pd.concat(out, ignore_index=True)



In [ ]:
# -----------------------
# 4) Categorical encoding (one-hot)
# -----------------------
cat_cols = [c for c in df.columns if df[c].dtype == "object"]
num_cols = [c for c in df.columns if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop",
)

pipe = Pipeline([("preprocess", preprocess)])

X = pipe.fit_transform(df)


In [ ]:
# Optional: feature names -> back to DataFrame
feature_names = pipe.named_steps["preprocess"].get_feature_names_out()
X_df = pd.DataFrame(X, columns=feature_names)

print("Final joined df shape:", df.shape)
print("Encoded matrix shape:", X_df.shape)


In [ ]:
import matplotlib.pyplot as plt

# pick one asset to inspect
asset = df["asset_id"].iloc[0]
d = df[df["asset_id"] == asset].sort_values("timestamp")

# 1) Missing values (top 15)
na = df.isna().sum().sort_values(ascending=False).head(15)
plt.figure()
na.plot(kind="bar")
plt.title("Top missing-value columns")
plt.tight_layout()
plt.show()


In [ ]:
# 2) Raw vs rolling (example columns if present)
for col in ["temperature", "vibration"]:
    roll = f"{col}_rollmean_5"
    if col in d.columns and roll in d.columns:
        plt.figure()
        plt.plot(d["timestamp"], d[col], label=col)
        plt.plot(d["timestamp"], d[roll], label=roll)
        plt.title(f"{asset}: {col} vs rolling mean")
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
# 3) Risk score distribution
if "risk_score" in df.columns:
    plt.figure()
    plt.hist(df["risk_score"].dropna(), bins=20)
    plt.title("Risk score distribution")
    plt.tight_layout()
    plt.show()


In [ ]:
# 4) Categorical cardinality (top 10 per cat column)
cat_cols = [c for c in df.columns if df[c].dtype == "object"]
for c in cat_cols[:5]:  # limit to first 5 to keep it quick
    plt.figure()
    df[c].value_counts(dropna=False).head(10).plot(kind="bar")
    plt.title(f"Top categories: {c}")
    plt.tight_layout()
    plt.show()


In [ ]:
# Write to SQLite

import sqlite3

DB_PATH = "etl_pipeline.db" # Database Object

with sqlite3.connect(DB_PATH) as conn:  # Keeps the whole process in a single connection context to prevent manual exit at end
    # Save joined + engineered data
    df.to_sql("etl_features_raw", conn, if_exists="replace", index=False)

    # Save encoded feature matrix
    X_df.to_sql("etl_features_encoded", conn, if_exists="replace", index=False)

    # (Optional) create indexes for faster lookup
    conn.execute("CREATE INDEX IF NOT EXISTS idx_raw_asset_time ON etl_features_raw(asset_id, timestamp)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_raw_asset ON etl_features_raw(asset_id)")



In [ ]:
# Verify by reading back

import pandas as pd
import sqlite3

with sqlite3.connect("etl_pipeline.db") as conn:
    df2 = pd.read_sql("SELECT * FROM etl_features_raw LIMIT 5", conn)
    X2  = pd.read_sql("SELECT * FROM etl_features_encoded LIMIT 5", conn)

print(df2.head())
print(X2.head())


In [ ]:

# Save to CSV (optional)
df.to_csv("final_joined.csv", index=False)
X_df.to_csv("final_features_encoded.csv", index=False)
